# Project

The aim of this project is to analyze the trend of the **crimes** reported to the authorities in **Italy** from `2006` to `2021`.

The dataset has been downloaded from **ISTAT** (***Italian National Institute of Statistics***) and is obtainable at the following link: http://dati.istat.it/Index.aspx?DataSetCode=DCCV_DELITTIPS#

To be more specific, this data focuses only on the **reported crimes committed during the reporting year**.

## Exploratory Data Analysis (EDA)

### Data Cleaning

We start the analysis by **importing** the needed **libraries** (*Numpy and Pandas*) and by storing the original `.csv` data in a **Pandas** **dataframe**

In [37]:
# Libraries needed
import pandas as pd
import numpy as np

# Importing data
original_df = pd.read_csv("data\DCCV_DELITTIPS_30052023105835453.csv")
df = original_df.copy()

By taking a quick look at the **first rows** of our data, with the `df.head()` method, we can see that there is a lot of **useless information** for what concernes our analysis.

In particular, I decided to drop the following **columns**: `"TIPO_DATO35"`, `"REATI_PS"`, `"RIF_TIME"`, `"Seleziona periodo"`, `"Flag Codes"`, `"Flags"`

In [23]:
df.head()

,ITTER107,Territorio,TIPO_DATO35,Tipo dato,REATI_PS,Tipo di delitto,RIF_TIME,Periodo del commesso delitto,TIME,Seleziona periodo,Value,Flag Codes,Flags
0,IT,Italia,CRIMEN,numero di delitti denunciati dalle forze di po...,MASSMURD,strage,YRDUR,durante l'anno di riferimento,2006,2006,26,NaN,NaN
1,IT,Italia,CRIMEN,numero di delitti denunciati dalle forze di po...,MASSMURD,strage,YRDUR,durante l'anno di riferimento,2007,2007,21,NaN,NaN
2,IT,Italia,CRIMEN,numero di delitti denunciati dalle forze di po...,MASSMURD,strage,YRDUR,durante l'anno di riferimento,2008,2008,28,NaN,NaN
3,IT,Italia,CRIMEN,numero di delitti denunciati dalle forze di po...,MASSMURD,strage,YRDUR,durante l'anno di riferimento,2009,2009,23,NaN,NaN
4,IT,Italia,CRIMEN,numero di delitti denunciati dalle forze di po...,MASSMURD,strage,YRDUR,durante l'anno di riferimento,2010,2010,12,NaN,NaN


In [38]:
# Dropping useless columns
df = df.drop(columns=['TIPO_DATO35', 'REATI_PS', 'RIF_TIME', 'Seleziona periodo', 'Flag Codes', 'Flags'])
df.head()

,ITTER107,Territorio,Tipo dato,Tipo di delitto,Periodo del commesso delitto,TIME,Value
0,IT,Italia,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2006,26
1,IT,Italia,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2007,21
2,IT,Italia,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2008,28
3,IT,Italia,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2009,23
4,IT,Italia,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2010,12


Now that we have a cleaner data, it's time to fix the **internal structure** of our dataset.

In particular:
* The data for the **province of Aosta** is listed as `"Valle d'Aosta / Vallée d'Aoste"`, the same name given to the **regional data**. In order to make the distinction more clear I decided to rename the provincial data as only `"Aosta"`;
* For a cleaner look, I also decided to rename the data of the **province of Bolzano** from `"Bolzano / Bozen"` to only `"Bolzano"`;
* Last, there was some **redundacy** about the data about the provinces of **Bolzano** and **Trento**, since they were listed in two different ways. After checking their validity, I choose to keep the rates listed with the codes `ITD10` (*Bolzano*) and `ITD20` (*Trento*) 

In [39]:
# Fixing Aosta province and Bolzano province name
df.loc[(df['ITTER107'] == 'ITC20'), 'Territorio'] = 'Aosta'
df.loc[(df['ITTER107'] == 'ITD10'), 'Territorio'] = 'Bolzano'

# For Bolzano/Bozen province, we keep the data with the itter107 == 'ITD10' and decide to delete the ITD1 data (they are the same)
df = df.loc[df['ITTER107'] != 'ITD1']

# For Trento province, we keep the data with the itter107 == 'ITD20' and decide to delete the ITD2 data (they are the same)
df = df.loc[df['ITTER107'] != 'ITD2']

Now we need to perform some more complex operation on the **internal structure** of our dataset. 
The data is listed under different levels:
* **Provincial**;
* **Regional**;
* **Macro-regional** (*north-west, north-east, centre, south, islands*);
* **National**.

This creates a lot of useless rows (***redundacy***), so we are gonna cut out all the data that is not on the **provincial** level, as we can also access the other levels by performing some simple methods like, for example, the **groupby**.

For this purpose I created a function that filters the rows we need from the ones we don't need.

In [40]:
def delete_regional_data(df):
    copy_df = df.copy()
    # List of regions and zones to delete from the df
    lista_regioni = [
        # Nord-ovest
        'Piemonte', 'Lombardia', 'Liguria', "Valle d'Aosta / Vallée d'Aoste",
        # Nord-est
        'Trentino Alto Adige / Südtirol', 'Friuli-Venezia Giulia', 'Veneto', 'Emilia-Romagna',
        # Centro
        'Toscana', 'Umbria', 'Marche', 'Lazio',
        # Sud
        'Abruzzo', 'Molise', 'Campania', 'Puglia', 'Basilicata', 'Calabria',
        # Isole
        'Sicilia', 'Sardegna']
    lista_zone = ['Italia', 'Nord-est', 'Nord-ovest', 'Centro', 'Sud', 'Isole']
    # For loop that iterates the df through every name in the lists we made before
    for region in lista_regioni:
        copy_df = copy_df.loc[(copy_df['Territorio'] != region)]
    for zona in lista_zone:
        copy_df = copy_df.loc[(copy_df['Territorio'] != zona)]
    return copy_df

df = delete_regional_data(df)

I decide to add two empty **columns**:
* `"Region"`
* `"Area"`

The idea is to create a function that adds, for each row, the specific **region** of each **province** and the specific **macro-area** of each **region**

In [41]:
# Adding columns at index 1
df.insert(loc=1, column='Region', value=np.nan)
df.insert(loc=1, column='Area', value=np.nan)

# Dictionary containing the various provinces
regioni_dict = {
    # Nord-ovest
    "Valle d'Aosta": ['Aosta'],
    'Piemonte': ['Torino', 'Vercelli', 'Biella', 'Verbano-Cusio-Ossola', 'Novara', 'Cuneo', 'Asti', 'Alessandria'],
    'Lombardia': ['Varese', 'Como', 'Lecco', 'Sondrio', 'Milano', 'Bergamo', 'Brescia', 'Pavia', 'Lodi', 'Cremona', 'Mantova', 'Monza e della Brianza'],
    'Liguria': ['Imperia', 'Savona', 'Genova', 'La Spezia'],

    # Nord-est
    'Trentino-Alto Adige / Südtirol': ['Bolzano', 'Trento'],
    'Veneto': ['Verona', 'Vicenza', 'Belluno', 'Treviso', 'Venezia', 'Padova', 'Rovigo'],
    'Friuli-Venezia Giulia': ['Pordenone', 'Udine', 'Gorizia', 'Trieste'],
    'Emilia Romagna': ['Piacenza', 'Parma', "Reggio nell'Emilia", 'Modena', 'Bologna', 'Ferrara', 'Ravenna', 'Forlì-Cesena', 'Rimini'],

    # Centro
    'Toscana': ['Massa-Carrara', 'Lucca', 'Pistoia', 'Firenze', 'Prato', 'Livorno', 'Pisa', 'Arezzo', 'Siena', 'Grosseto'],
    'Umbria': ['Perugia', 'Terni'],
    'Marche': ['Pesaro e Urbino', 'Ancona', 'Macerata', 'Ascoli Piceno', 'Fermo'],
    'Lazio': ['Viterbo', 'Rieti', 'Roma', 'Latina', 'Frosinone'],

    # Sud
    'Abruzzo': ["L'Aquila", 'Teramo', 'Pescara', 'Chieti'],
    'Molise': ['Isernia', 'Campobasso'],
    'Campania': ['Caserta', 'Benevento', 'Napoli', 'Avellino', 'Salerno'],
    'Puglia': ['Foggia', 'Bari', 'Taranto', 'Brindisi', 'Lecce', 'Barletta-Andria-Trani'],
    'Basilicata': ['Potenza', 'Matera'],
    'Calabria': ['Cosenza', 'Crotone', 'Catanzaro', 'Vibo Valentia', 'Reggio di Calabria'],

    # Isole
    'Sicilia': ['Trapani', 'Palermo', 'Messina', 'Agrigento', 'Caltanissetta', 'Enna', 'Catania', 'Ragusa', 'Siracusa'],
    'Sardegna': ['Sassari', 'Nuoro', 'Cagliari', 'Oristano']
}

# Dictionary containing the various regions
area_dict = {
    'Nord-ovest': ["Valle d'Aosta", 'Piemonte', 'Lombardia', 'Liguria'],
    'Nord-est': ['Trentino-Alto Adige / Südtirol', 'Veneto', 'Friuli-Venezia Giulia', 'Emilia Romagna'],
    'Centro': ['Toscana', 'Umbria', 'Marche', 'Lazio'],
    'Sud': ['Abruzzo', 'Molise', 'Campania', 'Puglia', 'Basilicata', 'Calabria'],
    'Isole': ['Sicilia', 'Sardegna']
}

def add_region_and_zone_columns(df):
    df = df.copy()
    # Filling 'Region' column based on the various provinces
    regions = list(regioni_dict.keys())
    for region in regions:
        df.loc[(df['Territorio'].isin(regioni_dict[region])), 'Region'] = region
    # Filling 'Area' column based on the various regions
    areas = list(area_dict.keys())
    for area in areas:
        df.loc[(df['Region'].isin(area_dict[area])), 'Area'] = area
    return df

df = add_region_and_zone_columns(df)
df

,ITTER107,Area,Region,Territorio,Tipo dato,Tipo di delitto,Periodo del commesso delitto,TIME,Value
2676,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2006,0
2677,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2007,2
2678,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2008,3
2679,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2009,0
2680,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2010,1
...,...,...,...,...,...,...,...,...,...
118683,ITG12,Isole,Sicilia,Palermo,numero di delitti denunciati dalle forze di po...,usura,durante l'anno di riferimento,2017,6
118684,ITG12,Isole,Sicilia,Palermo,numero di delitti denunciati dalle forze di po...,usura,durante l'anno di riferimento,2018,5
118685,ITG12,Isole,Sicilia,Palermo,numero di delitti denunciati dalle forze di po...,usura,durante l'anno di riferimento,2019,6
118686,ITG12,Isole,Sicilia,Palermo,numero di delitti denunciati dalle forze di po...,usura,durante l'anno di riferimento,2020,8


Last, after all the slicing perfomed, we reset the index of our dataset and create a copy of it

In [42]:
# Reset index
df = df.reset_index().drop(columns='index')

# Creating a copy of the cleaned dataset
cleaned_original_df = df.copy()

# Snippet of the cleaned dataset
df.head()

,ITTER107,Area,Region,Territorio,Tipo dato,Tipo di delitto,Periodo del commesso delitto,TIME,Value
0,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2006,0
1,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2007,2
2,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2008,3
3,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2009,0
4,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2010,1


### Adding Population information

The crime rates we see in the dataset we just cleaned are clearly biased by the **population** of each city: bigger cities will tend to have higher rates than smaller cities. 

In order to try to limit this **population bias** I decided to compute the **incidence of the rates per 100,000 inhabitants**.

To compute this value we need to get the data concerning the population, which can be found in the ISTAT website at the following links
* http://dati.istat.it/Index.aspx?DataSetCode=DCIS_RICPOPRES2011 (*contains data from 2001 to 2019*);
* http://dati.istat.it/Index.aspx?DataSetCode=DCIS_POPRES1 (*contains data for 2020 and 2021*)

In [15]:
# Importing population data
pop_df_06_19 = pd.read_csv('data\DCIS_RICPOPRES2011_20062023153653173.csv')
pop_df_2020 = pd.read_csv('data\DCIS_POPRES1_20062023160226593.csv')

In [19]:
pop_df_06_19.head()

,ITTER107,Territorio,TIPO_DATO15,Tipo dato,ETA1,Classe di età,SEXISTAT1,Sesso,CITTADINANZA,Cittadinanza,TIME,Seleziona periodo,Value,Flag Codes,Flags
0,ITF3,Campania,JAN,popolazione al 1º gennaio,TOTAL,totale,1,maschi,ITL,italiano-a,2002,2002,2759224,NaN,NaN
1,ITF3,Campania,JAN,popolazione al 1º gennaio,TOTAL,totale,1,maschi,ITL,italiano-a,2003,2003,2759657,NaN,NaN
2,ITF3,Campania,JAN,popolazione al 1º gennaio,TOTAL,totale,1,maschi,ITL,italiano-a,2004,2004,2761076,NaN,NaN
3,ITF3,Campania,JAN,popolazione al 1º gennaio,TOTAL,totale,1,maschi,ITL,italiano-a,2005,2005,2763955,NaN,NaN
4,ITF3,Campania,JAN,popolazione al 1º gennaio,TOTAL,totale,1,maschi,ITL,italiano-a,2006,2006,2762664,NaN,NaN


In [18]:
pop_df_2020.head()

,ITTER107,Territorio,TIPO_DATO15,Tipo di indicatore demografico,SEXISTAT1,Sesso,ETA1,Età,STATCIV2,Stato civile,TIME,Seleziona periodo,Value,Flag Codes,Flags
0,IT,Italia,JAN,popolazione al 1º gennaio,9,totale,Y0,0 anni,1,nubile/celibe,2020,2020,414974,NaN,NaN
1,IT,Italia,JAN,popolazione al 1º gennaio,9,totale,Y0,0 anni,1,nubile/celibe,2021,2021,404956,NaN,NaN
2,IT,Italia,JAN,popolazione al 1º gennaio,9,totale,Y0,0 anni,99,totale,2020,2020,414974,NaN,NaN
3,IT,Italia,JAN,popolazione al 1º gennaio,9,totale,Y0,0 anni,99,totale,2021,2021,404956,NaN,NaN
4,ITC,Nord-ovest,JAN,popolazione al 1º gennaio,9,totale,Y0,0 anni,1,nubile/celibe,2020,2020,109812,NaN,NaN


Population data from `2006` to `2019` is divided for **male** and **female** and also for **italian** and **foreign** citizenship.

We need to locate in the dataframe only the **total values**.

For the other dataset containing the population data for `2019` and `2020` the situation is a bit different as the data is divided over **age** and **marital status**. Nonetheless, we still need to locate in the datagrame only the **total values**. 

In [20]:
# 2006 - 2019 data
pop_df_06_19 = pop_df_06_19.loc[(pop_df_06_19['Sesso'] == 'totale') & (pop_df_06_19['Cittadinanza'] == 'totale')]
# 2020 - 2021 data
pop_df_2020 = pop_df_2020.loc[(pop_df_2020['Età'] == 'totale') & (pop_df_2020['Stato civile'] == 'totale')]

Once we filtered out all the useless data, it is time to drop the **columns** we don't need from both datasets

In [22]:
pop_df_06_19 = pop_df_06_19.drop(columns=['TIPO_DATO15', 'Tipo dato' ,'ETA1', 'SEXISTAT1', 'Sesso', 'Classe di età','Cittadinanza', 'CITTADINANZA', 'Seleziona periodo', 'Flag Codes', 'Flags'])

pop_df_2020 = pop_df_2020.drop(columns=['TIPO_DATO15', 'Tipo di indicatore demografico', 'SEXISTAT1', 'Sesso', 'ETA1', 'STATCIV2', 'Seleziona periodo', 'Flag Codes', 'Flags', 'Età', 'Stato civile'])


Now, since the population data we downloaded is from **ISTAT**, the same page as the crime rates data we worked with before, we need to perform on these new two datasets the same cleaning process. In particular:
* Renaming the data for the provinces of Aosta and Bolzano;
* Cleaning the redundacy about the provinces of Trento and Bolzano;
* Also, this time we need to delete the population data regarding the province of **Sud Sardegna**. We do this because we don't have any crime rate data for this province, so this move will avoid any future null values in the final merged data;
* Last, we fix the redundacy of the data by deleting all **non-regional** data with the function created previously.

In [26]:
# Fixing Aosta province and Bolzano province name
# 2006 - 2019 data
pop_df_06_19.loc[(pop_df_06_19['ITTER107'] == 'ITC20'), 'Territorio'] = 'Aosta'
pop_df_06_19.loc[(pop_df_06_19['ITTER107'] == 'ITD10'), 'Territorio'] = 'Bolzano'
# 2020 and 2021 data
pop_df_2020.loc[(pop_df_2020['ITTER107'] == 'ITC20'), 'Territorio'] = 'Aosta'
pop_df_2020.loc[(pop_df_2020['ITTER107'] == 'ITD10'), 'Territorio'] = 'Bolzano'

# For Bolzano/Bozen province, we keep the data with the itter107 == 'ITD10' and decide to delete the ITD1 data (they are the same)
# 2006 - 2019 data
pop_df_06_19 = pop_df_06_19.loc[pop_df_06_19['ITTER107'] != 'ITD1']
# 2020 and 2021 data
pop_df_2020 = pop_df_2020.loc[pop_df_2020['ITTER107'] != 'ITD1']

# For Trento province, we keep the data with the itter107 == 'ITD20' and decide to delete the ITD2 data (they are the same)
# 2006 - 2019 data
pop_df_06_19 = pop_df_06_19.loc[pop_df_06_19['ITTER107'] != 'ITD2']
# 2020 and 2021 data
pop_df_2020 = pop_df_2020.loc[pop_df_2020['ITTER107'] != 'ITD2']

# Deleting Sud Sardegna (there is no data for this province in the other dataset)
# 2006 - 2019 data
pop_df_06_19 = pop_df_06_19.loc[pop_df_06_19['Territorio'] != 'Sud Sardegna']
# 2020 and 2021 data
pop_df_2020 = pop_df_2020.loc[pop_df_2020['Territorio'] != 'Sud Sardegna']

# Deleting all non-regional data as we did before
# 2006 - 2019 data
pop_df_06_19 = delete_regional_data(pop_df_06_19)
# 2020 and 2021 data
pop_df_2020 = delete_regional_data(pop_df_2020)

Now that the two datasets are cleaned, last step is to **concatenate** them both into a single dataset.

In [27]:
pop_df = pd.concat([pop_df_06_19, pop_df_2020], ignore_index=True)
pop_df.head()

,ITTER107,Territorio,TIME,Value
0,ITC11,Torino,2002,2165695
1,ITC11,Torino,2003,2169876
2,ITC11,Torino,2004,2189701
3,ITC11,Torino,2005,2202292
4,ITC11,Torino,2006,2208364


Since the `"Value"` column already exists in the other crime rate dataset, we rename this column in the population data to `"Population"`

In [28]:
pop_df = pop_df.rename(columns= {'Value' : 'Population'})

Final step is to **merge** the population data with the crime rates data into a single dataframe.

In [29]:
final_df = pd.merge(df, pop_df, on=('ITTER107', 'Territorio', 'TIME'), how='left')
final_df.head()

,ITTER107,Area,Region,Territorio,Tipo dato,Tipo di delitto,Periodo del commesso delitto,TIME,Value,Population
0,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2006,0,2208364
1,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2007,2,2214122
2,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2008,3,2245170
3,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2009,0,2260921
4,ITC11,Nord-ovest,Piemonte,Torino,numero di delitti denunciati dalle forze di po...,strage,durante l'anno di riferimento,2010,1,2268453


### Adding incidence of the rates over `100'000` population

Once we obtained a dataframe with both population and crime rates information, in order to add the incidence of the rates over `100'000` population we need to perform some simple operation:
* First, we create an empty column called "`"Value over 100000 population"`";
* We create a function that, for each row of the dataset, **computest** the **incidence** and stores it in the empty column just created.

In [ ]:
# Adding empty column
final_df.insert(loc=9, column='Value over 100000 population', value=np.nan)

# Function that iterates through each row of the data and computes the value of the incidence, storing it into the empty column just created
def value_over_100k(df):
    i = 0
    for i in range(len(df.index)):
        value = df.iloc[i]['Value']
        population = df.iloc[i]['Population']
        value_over_100k = round(((value / population) * 100000), 4)
        df['Value over 100000 population'][i] = value_over_100k
    return df

final_df = value_over_100k(final_df)
final_df.head()

Last, before saving our final dataset, we drop the `"Tipo dato"` and `"Periodo del commesso delitto"` columns, for a cleaner look.

Also, we **rename** the columns from **italian** to **english**.

In [31]:
# Dropping columns
final_df = final_df.drop(columns=['Tipo dato', 'Periodo del commesso delitto'])

# Renaming
final_df = final_df.rename(columns={'Territorio' : 'Province', 'Tipo di delitto' : 'Crime', 'TIME' : 'Year'})

final_df.head()

,ITTER107,Area,Region,Province,Crime,Year,Value,Value over 100000 population,Population
0,ITC11,Nord-ovest,Piemonte,Torino,strage,2006,0,0.0000,2208364
1,ITC11,Nord-ovest,Piemonte,Torino,strage,2007,2,0.0903,2214122
2,ITC11,Nord-ovest,Piemonte,Torino,strage,2008,3,0.1336,2245170
3,ITC11,Nord-ovest,Piemonte,Torino,strage,2009,0,0.0000,2260921
4,ITC11,Nord-ovest,Piemonte,Torino,strage,2010,1,0.0441,2268453


Last, the final version of the dataset is saved as a csv file in order to make future access easier

In [ ]:
final_df.to_csv("data/final.csv")

### Shapefile and GeoJSON

In this section some work on **shapefiles** is performed
These files are going to be used in the **streamlit dashboard** with the **choropleth map** built with **Plotly**

Since Plotly maps need a JSON file in order to work, we need to convert our SHP file

In [ ]:
import geopandas as gdp
import json

# Reading .shp file with GeoPandas
file_path = "data\shapefiles\ProvCM01012023_g\ProvCM01012023_g_WGS84.shp"
shp_prov = gdp.read_file(file_path)

# Brief cleaning in order to match the information with the main dataframe
shp_prov = shp_prov.loc[shp_prov['DEN_UTS'] != 'Sud Sardegna']
shp_prov.loc[shp_prov['SIGLA'] == 'MS', 'DEN_UTS'] = 'Massa-Carrara'
final_df.loc[final_df['Province'] == 'Forlì-Cesena', 'Province'] = "Forli'-Cesena"

# Converting to 4326
shp_prov = shp_prov.to_crs(4326)
# Converting into JSON file
shp_prov.to_file('data\shapefiles\province.geojson', driver='GeoJSON')

shp_prov.head()